Pytorch - один из самых популярных фреймворков глубокого обучения для ML-специалистов. Фактически сегодня это еще и целая [экосистема](https://pytorch.org/ecosystem/) инструментов

В библиотеке есть четыре ключевых составляющих:

- Развитый инструментарий для работы с тензорами. Он похож на numpy, но даёт дополнительные возможности по контролю выделяемой памяти, что важно при работе с большими моделями и данными.
- Простое построение динамического вычислительного графа, позволяющего получать градиенты целевых функций от параметров модели.
- Большой набор готовых слоёв для построения нейронных сетей произвольной архитектуры.
- Возможность перенаправлять вычисления на графические процессоры GPU.



In [ ]:
import torch

### Тензоры


Создание тензора

In [ ]:
x = torch.empty(5, 3)
print(x)

tensor([[-6.7967e-07,  4.5666e-41, -7.3119e-07],
        [ 4.5666e-41, -5.3389e-07,  4.5666e-41],
        [-7.0882e-07,  4.5666e-41, -7.0877e-07],
        [ 4.5666e-41, -7.2292e-07,  4.5666e-41],
        [-7.2081e-07,  4.5666e-41, -6.8812e-07]])


Случайная инициализация в диапазоне [0; 1]

In [ ]:
x = torch.rand(5, 3)
print(x)

tensor([[0.8341, 0.0180, 0.0352],
        [0.6997, 0.8201, 0.6471],
        [0.9095, 0.4144, 0.7317],
        [0.8288, 0.3143, 0.1800],
        [0.3573, 0.5910, 0.7238]])


Инициализация нулями

In [ ]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


Создание непосредственно из данных

In [ ]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


Из другого тензора

In [ ]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # the same size as input that is filled with random numbers from a normal distribution, override dtype!
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-0.4060, -0.0181, -0.6774],
        [-0.4574, -0.8045,  1.3948],
        [ 2.3655, -0.2676,  0.3849],
        [ 1.0001, -1.4454,  0.8617],
        [ 0.2797,  0.2810, -1.5000]])


При преобразовании типа (если он меняется) под данные выделяется новая память

In [ ]:
x = torch.Tensor(5, 3)
y = x.long()
y = x.float()

Размер тензора

In [ ]:
x.size()

torch.Size([5, 3])

In [ ]:
x.shape

torch.Size([5, 3])

NB! torch.Size - абстракция от tuple, поэтому поддерживаются те же операции, как и с кортежами

### Операции с тензорами


In [ ]:
y = torch.rand(5, 3)
print(x + y)

tensor([[0.4233, 0.1546, 0.8740],
        [0.1989, 0.2291, 0.6066],
        [0.6610, 0.2598, 0.6111],
        [0.5708, 0.1070, 0.4649],
        [0.9717, 0.0444, 0.9573]])


In [ ]:
print(torch.add(x, y))

tensor([[0.4233, 0.1546, 0.8740],
        [0.1989, 0.2291, 0.6066],
        [0.6610, 0.2598, 0.6111],
        [0.5708, 0.1070, 0.4649],
        [0.9717, 0.0444, 0.9573]])


Выходная переменная как параметр

In [ ]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[0.4233, 0.1546, 0.8740],
        [0.1989, 0.2291, 0.6066],
        [0.6610, 0.2598, 0.6111],
        [0.5708, 0.1070, 0.4649],
        [0.9717, 0.0444, 0.9573]])


in-place операции - operation_ syntax, новая память не выделяется

In [ ]:
x.add(y)

tensor([[0.4233, 0.1546, 0.8740],
        [0.1989, 0.2291, 0.6066],
        [0.6610, 0.2598, 0.6111],
        [0.5708, 0.1070, 0.4649],
        [0.9717, 0.0444, 0.9573]])

In [ ]:
x.add_(y)

tensor([[0.4233, 0.1546, 0.8740],
        [0.1989, 0.2291, 0.6066],
        [0.6610, 0.2598, 0.6111],
        [0.5708, 0.1070, 0.4649],
        [0.9717, 0.0444, 0.9573]])

Синтаксический сахар NumPy индексации

In [ ]:
print(x[:, 1])

tensor([0.1546, 0.2291, 0.2598, 0.1070, 0.0444])


Форма тензора (число индексов и их размерности) меняется функциями `view` и `reshape`

In [ ]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8) # Одна из размерностей может быть равна -1 и тогда она будет посчитана автоматически
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


`view` - создает другое представление исходного тензора. При изменении формы `view` меняется `x.stride()`. Новый тензор всегда делит (share) данные с исходным тензором

`reshape` не дает гарантии на шэринг данных: сначала проверяется `is_contiguous` и если результа - False, вызывается `contiguous` (создаёт новую память). После этого вызывается `view`

In [ ]:
x = torch.randn(4, 4)
y = x.reshape(16)
z = x.reshape(-1, 8)
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


Получение значения тензора из одного элемента

In [ ]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([1.9939])
1.993883490562439


In [ ]:
y[1].item()

-0.02648058533668518

Проверка доступности GPU и создание тензора на GPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))

tensor([2.9939], device='cuda:0')
tensor([2.9939], dtype=torch.float64)


Чтобы не прописывать device руками

In [ ]:
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
device

'cuda:0'

Выигрыш во времени от использования GPU

In [ ]:
%%time

x1 = torch.eye(10000)
y1 = torch.eye(10000)
z1 = x1.mm(y1)

CPU times: user 27 s, sys: 709 ms, total: 27.7 s
Wall time: 28.1 s


In [ ]:
%%time

x1 = torch.eye(10000, device=device)
y1 = torch.eye(10000, device=device)
z1 = x1.mm(y1)

CPU times: user 431 ms, sys: 265 ms, total: 696 ms
Wall time: 2.78 s


### Autograd - automatic differentiation engine

[PyTorch 101, Part 1: Understanding Graphs, Automatic Differentiation and Autograd](https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/)

В центре большинства современных приемов машинного обучения лежит расчет градиентов. Это в особенности касается нейронных сетей, где для обновления весовых коэффициентов используется алгоритм обратного распространения

Autograd предоставляет классы и функции, реализующие автоматическое дифференцирование произвольных скалярных функций. Это требует минимальных изменений в существующем коде - нужно только объявить Tensor, для которого должны вычисляться градиенты, с атрибутом `requires_grad=True`

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [ ]:
x.grad == None

True

In [ ]:
x.grad_fn == None

True

После применения какой-либо операции к тензору атрибуту `grad_fn`  присваивается объект `Function`, который добавляется в граф вычислений для обратного распространения градиента.



In [ ]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [ ]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.grad_fn` может менять "на лету"

In [ ]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


Метод `backward` корневого узла графа вычислений запускает процедуру вычисления градиентов в листовых (is_leaf) узлах, имеющих атрибут requires_grad. Граф дифференцируется по цепочке (chain rule)

In [ ]:
out.backward()

In [ ]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


По умолчанию промежуточные (не листовые) узлы графа не хранят прошедшие через них градиентов.

In [ ]:
print(y.grad)

None


<ipython-input-32-b81046718426>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:486.)
  print(y.grad)


Эту ситуацию можно изменить, вызвав для для конкретного узла метод retain_grad

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
y.retain_grad()
z = y * y * 3
out = z.mean()
out.backward()

In [ ]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [ ]:
print(y.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


Иногда с листовыми узлами необходимо проделать действия, не меняя при этом графа. Такие действия проводят, используя контекстный менедежр `no_grad`, которое блокирует создание новых узлов графа

In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad(): # потом можно включить вручную torch.enable_grad()
    print((x ** 2).requires_grad)


True
True
False
